In [ ]:
import numpy as np
from scipy import signal

import math

import matplotlib.pyplot as plt
from tqdm import tqdm

%load_ext autoreload
%autoreload 2

TODO:
- check what happens in g_vector for t = 0
- check all code for general mistakes

This notebook simulates the rough Heston model as introduced by Guennoun, Jacquier, Roome and Shi in <cite data-cite="guennoun2018asymptotic">Guennoun, Hamza, et al. "Asymptotic behavior of the fractional Heston model." SIAM Journal on Financial Mathematics 9.3 (2018): 1017-1045.</cite>. In particular it generates a rough volatility. 

In [ ]:
n = 1000 # Number of timesteps
sqrtn = math.sqrt(n) 
M = 1 # Number of simulations
rho = 1 # Correlation between the normal distributions dzeta and eta
T = 1 # Total time
alpha = -0.25 # Alpha as in G^\alpha. It has to lie in (-.5, .5) to have any meaning. For negative alphas, you get roughness

# Some volatility parameters
kappa = 1
theta = 10 # 
vola_0 = 10 # initial value of the non-rough vola
eta = 1
ksi = 1
X_0 = 1 # initial value of X, the log-stock price 

In [ ]:
def g(t):
    return t**alpha
g_vector = np.fromfunction(lambda i: g(T*i/n), shape=(n,), dtype=int)
g_vector[0] = 0

In [ ]:
def b(y):
    return kappa * (theta-y)
def a(y):
    return ksi * math.sqrt(y)
def Phi(y):
    return eta + y

The Notebook first simulates the volatility according to
$$
dV_t = \kappa(\theta - V_t)dt + \xi \sqrt{V_t}dW_t
$$

In [ ]:
vola = np.zeros((n,)) # The discretised non-rough volatility process
Delta_vola = np.zeros((n,)) # vola_i - vola_{i-1}

In [ ]:
W = np.random.normal(0.0, 1.0, size=(n,)) # The unscaled brownian motion
vola[0] = vola_0
for i in range(1, n):
    vola[i] = vola[i-1] + (T/n)*b(vola[i-1]) + (T/sqrtn)*a(vola[i-1])*W[i-1]
    Delta_vola[i] = vola[i] - vola[i - 1]

Using Fast Fourier Transform (FFT), the volatility is made rough. For more details, see <cite data-cite="horvath2017functional">Horvath, Blanka, Antoine Jack Jacquier, and Aitor Muguruza. "Functional central limit theorems for rough volatility." Available at SSRN 3078743 (2017)</cite>

In [ ]:
# For some unknown reason, this doesn't work, but a valid solution was found
# fourier_transformed_G_alpha = np.fft.fft(g_vector)*np.fft.fft(Delta_Y) 
# G_alpha_Y = np.fft.ifft(fourier_transformed_G_alpha) # This is supposed to the the rough volatility 

In [ ]:
# To compare it with the theoretical value
# G_a_Y = np.zeros((n,))
# for i in range(n):
#     for k in range(1, i+1):
#         G_a_Y[i] += g_vector[i-k + 1] * Delta_Y[k] 
# G_a_Y

In [ ]:
rough_vola = signal.fftconvolve(g_vector, Delta_vola, 'same')

<!--bibtex

@article{guennoun2018asymptotic,
  title={Asymptotic behavior of the fractional Heston model},
  author={Guennoun, Hamza and Jacquier, Antoine and Roome, Patrick and Shi, Fangwei},
  journal={SIAM Journal on Financial Mathematics},
  volume={9},
  number={3},
  pages={1017--1045},
  year={2018},
  publisher={SIAM}
}

@article{horvath2017functional,
  title={Functional central limit theorems for rough volatility},
  author={Horvath, Blanka and Jacquier, Antoine Jack and Muguruza, Aitor},
  journal={Available at SSRN 3078743},
  year={2017}
}

-->

Using this rough volatility, a standard simulation is run on the log prices, based on the equation

$$
dX_t = -\frac{1}{2}V_tdt + \sqrt{V_t} dB_t
$$

In [ ]:
X = np.zeros((M, n))
for j in range(M):
    # Create correlated Brownian motion
    randoms = np.random.normal(0.0, 1.0, size=(n,))
    B = rho*W + math.sqrt(1 - rho**2) * randoms
    X[j, 0] = X_0
    for i in range(1, n):
        X[j, i] = X[j, i-1] - (1/2)*(T/n)*Phi(rough_vola[i-1]) + math.sqrt((T/n) * Phi(rough_vola[i-1])) * B[i-1]

In [ ]:
# A function which gives the brownian motion used to generate the rough volatility and the volatility.
def rough_volatility():
    vola = np.zeros((n,))
    Delta_vola = np.zeros((n,))
    W = np.random.normal(0.0, 1.0, size=(n,))
    g_vector = np.fromfunction(lambda i: g(T*i/n), shape=(n,), dtype=int)
    vola[0] = vola_0
    for i in range(1, n):
        vola[i] = vola[i-1] + (T/n)*b(vola[i-1]) + (T/sqrtn)*a(vola[i-1])*W[i-1]
        Delta_vola[i] = vola[i] - vola[i - 1]
    rough_vola = signal.fftconvolve(g_vector, Delta_vola, 'same')
    return (W, G_alpha_Y)

In [ ]:
print(X)

In [ ]:
plt.plot(rough_vola)